In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install jcopml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,Normalizer,MinMaxScaler,RobustScaler,PolynomialFeatures,PowerTransformer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
from xgboost import XGBClassifier
from skopt.space import Integer,Real

from jcopml.plot import plot_correlation_matrix,plot_classification_report,plot_confusion_matrix,plot_residual,plot_association_matrix
from jcopml.tuning import grid_search_params as gsp, random_search_params as rsp, bayes_search_params as bsp
from jcopml.feature_importance import mean_score_decrease
from skopt import BayesSearchCV
from jcopml.tuning.space import Integer,Real

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
df.head()

# EDA

In [ ]:
# Check NaN values
df.isna().sum()

In [ ]:
# Check Shape Of Dataframe
df.shape

In [ ]:
# Check and Drop Duplicate
duplicate_df = df[df.duplicated()]
print(" number of duplicate rows: ", duplicate_df.shape )

df = df.drop_duplicates()
df.shape

In [ ]:
# Check Outliers
plt.figure(figsize=(40,20))
sns.boxplot(data=df);

In [ ]:
#removing Out liers 
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape

In [ ]:
df['label_quality'] = pd.cut(df['quality'],[2,6,10],labels=[0,1])
df['label_quality']=df['label_quality'].astype('int')
df.head()

In [ ]:
sns.barplot(x='label_quality',y='alcohol',data=df);

In [ ]:
pd.value_counts(df['label_quality']).plot.bar()
plt.title('Quality Bar Plot')
plt.xlabel('Quality')
plt.ylabel('Frequency')
df['label_quality'].value_counts()

In [ ]:
plt.figure(figsize=(13,6))
plt.subplot(121)
sns.scatterplot("sulphates","alcohol","label_quality",data=df);

# Splitting Data

In [ ]:
X = df.drop(columns=['quality','label_quality'])
y = df['label_quality']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

# Whitout SMOTE

In [ ]:
X = df.drop(columns=['quality','label_quality'])
y = df['label_quality']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
num_pipe = Pipeline([("scaler",RobustScaler()),
                    ("transform",PowerTransformer(method='yeo-johnson'))])

preprosesor = ColumnTransformer([("numeric",num_pipe,X_train.columns)])

pipeline = Pipeline([("prep",preprosesor),
                    ("ml",XGBClassifier(n_jobs=-1,random_state=42))])

parameter = {'ml__max_depth': Integer(low=8, high=12),
 'ml__learning_rate': Real(low=-2, high=0, prior='log-uniform'),
 'ml__n_estimators': Integer(low=150, high=250),
 'ml__subsample': Real(low=0.5, high=0.8, prior='uniform'),
 'ml__gamma': Integer(low=1, high=5),
 'ml__colsample_bytree': Real(low=0.5, high=1, prior='uniform'),
 'ml__reg_alpha': Real(low=-3, high=1, prior='log-uniform'),
 'ml__reg_lambda': Real(low=-3, high=1, prior='log-uniform')}

model = RandomizedSearchCV(pipeline,parameter,cv=3,n_iter=150,scoring='f1',n_jobs=-1,verbose=1)
model.fit(X_train,y_train)

# Evaluation
print(model.best_params_)
print(model.score(X_train,y_train),model.best_score_,model.score(X_test,y_test))

In [ ]:
plot_confusion_matrix(X_train, y_train, X_test, y_test, model)

In [ ]:
# Classification Report
plot_classification_report(X_train, y_train, X_test, y_test, model, report=True)

In [ ]:
# Feature Importance
df_imp = mean_score_decrease(X_train, y_train, model, plot=True, topk=10)

# With SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X = df.drop(columns=['quality','label_quality'])
y = df['label_quality']

X.head()

In [ ]:
sm = SMOTE(random_state=42)
X_sm, y_sm = sm.fit_resample(X, y)

print(f"Before Resampling {X.shape}, After Resampling {X_sm.shape}")

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_sm,y_sm,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
num_pipe = Pipeline([("scaler",RobustScaler()),
                    ("transform",PowerTransformer(method='yeo-johnson'))])

preprosesor = ColumnTransformer([("numeric",num_pipe,X_train.columns)])

pipeline = Pipeline([("prep",preprosesor),
                    ("ml",XGBClassifier(n_jobs=-1,random_state=42))])

parameter = {'ml__max_depth': Integer(low=8, high=12),
 'ml__learning_rate': Real(low=-2, high=0, prior='log-uniform'),
 'ml__n_estimators': Integer(low=150, high=250),
 'ml__subsample': Real(low=0.5, high=0.8, prior='uniform'),
 'ml__gamma': Integer(low=1, high=5),
 'ml__colsample_bytree': Real(low=0.5, high=1, prior='uniform'),
 'ml__reg_alpha': Real(low=-3, high=1, prior='log-uniform'),
 'ml__reg_lambda': Real(low=-3, high=1, prior='log-uniform')}

model = RandomizedSearchCV(pipeline,parameter,cv=3,n_iter=150,scoring='f1',n_jobs=-1,verbose=1)
model.fit(X_train,y_train)

# Evaluation
print(model.best_params_)
print(model.score(X_train,y_train),model.best_score_,model.score(X_test,y_test))

In [ ]:
plot_confusion_matrix(X_train, y_train, X_test, y_test, model)

In [ ]:
# Classification Report
plot_classification_report(X_train, y_train, X_test, y_test, model, report=True)

In [ ]:
# Feature Importance
df_imp = mean_score_decrease(X_train, y_train, model, plot=True, topk=10)